
**Student of EVA7 Batch awaiting EVA Phase II submitting EVA8 Transformer Assignments** </br>

Repository github url : https://github.com/jai-mr/Session </br>

Assignment Repository : https://github.com/jai-mr/Session/blob/main/S11/README.md</br>

Submitted by : Jaideep R - No Partners</br>

Registered email id : jaideepmr@gmail.com</br>


**Objective: Code for Sparse attention on GPT ***</br>
1. Implement sparse attention in the GPT Code
2. Train on custom data collected for training BERT in the first part of the assignment
3. Share training logs and 10 examples of output

In [1]:
# Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
from google.colab import drive
drive.mount("/content/gdrive",force_remount=True)
gdrivepath="/content/gdrive/MyDrive/EVA8/s11/"

import os 
os.chdir(gdrivepath+'s11/')

Mounted at /content/gdrive


In [14]:
%pwd

'/content/gdrive/MyDrive/EVA8/s11/s11'

In [3]:
%ls

BERT/  model.py      ReadMe.md       S11_GPT.ipynb    utils.py
GPT/   __pycache__/  S11_BERT.ipynb  S11_GPT_o.ipynb


In [2]:
os.chdir(gdrivepath+'s11/')
%pwd
%ls

BERT/  model.py   S11_BERT.ipynb  S11_GPT_o.ipynb
GPT/   ReadMe.md  S11_GPT.ipynb   utils.py


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#os.chdir(gdrivepath+'s11/GPT/')
%pwd
%ls

BERT/  model.py   S11_BERT.ipynb  S11_GPT_o.ipynb
GPT/   ReadMe.md  S11_GPT.ipynb   utils.py


## Import Dataset, Model & Device

In [5]:
import torch
from transformers import AutoTokenizer  # pip install transformers
from model import Transformer

from utils import (
    BATCH_SIZE,
    BLOCK_SIZE,
    DEVICE,
    DROPOUT,
    LEARNING_RATE,
    NUM_EMBED,
    NUM_HEAD,
    NUM_LAYER,
    MAX_ITER,
    EVAL_INTER,
    encode,
    decode,
    get_batch,
    save_model_to_chekpoint,
    estimate_loss,
)


In [6]:
DEVICE

'cpu'

In [7]:
torch.cuda.is_available()

False

In [8]:
%pwd
%ls /content/gdrive/MyDrive/EVA8/s11/s11/GPT/GPT_Dataset_file.txt

/content/gdrive/MyDrive/EVA8/s11/s11/GPT/GPT_Dataset_file.txt


In [10]:
# load model from checkpoint
# m = load_model_from_checkpoint(Transformer,vocab_size=vocab_size)

# example to decode sequence
# enc_sec = m.generate(idx=torch.zeros((1,1), dtype=torch.long),
# max_new_tokens=20)[0].tolist()
# print(decode(vocab=vocab, enc_sec=enc_sec))

# raw data
#path_do_data = "data/english.txt"
path_do_data = "/content/gdrive/MyDrive/EVA8/s11/s11/GPT/GPT_Dataset_file.txt"
data_raw = open(path_do_data, encoding="utf-8").read()
# we use pretrained BERT tokenizer for performance improvements
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
# data_raw = data_raw[4000000:] # short dataset

# train/val split
data = encode(text_seq=data_raw, tokenizer=tokenizer)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]



Token indices sequence length is longer than the specified maximum sequence length for this model (13562746 > 512). Running this sequence through the model will result in indexing errors


In [11]:
# train a new model
model = Transformer(
    vocab_size=vocab_size,
    num_embed=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
)
# load model to GPU if available
m = model.to(DEVICE)
# print the number of parameters in the model
print(
    "Model with {:.2f}M parameters".format(sum(p.numel() for p in m.parameters()) / 1e6)
)


Model with 89.48M parameters


## Define Optimizer and Train

In [12]:
# optimizer takes the model's parameters and the learning rate as input,
# and updates the parameters during the training process in order to
# minimize the loss function.
optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)
MAX_ITER = 500
EVAL_INTER = 50
for step in range(MAX_ITER):

    # every EVAL_INTER evaluate the loss on train and val sets
    if step % EVAL_INTER == 0 or step == MAX_ITER - 1:
        loss_train = estimate_loss(
            data=train_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        loss_val = estimate_loss(
            data=val_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        print("step {:10} | train loss {:6.4f} | val loss {:6.4f}".format(step, loss_train, loss_val))

    # sample a batch of data
    xb, yb = get_batch(data=train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    logits, loss = m.forward(xb, yb)
    # zero_grad() method sets the gradients of all parameters in the optimizer to zero
    optimizer.zero_grad(set_to_none=True)
    # backward() method on the loss variable calculates the gradients 
    # of the loss with respect to the model's parameters.
    loss.backward()
    # step() method on the optimizer updates the model's parameters 
    # using the calculated gradients, in order to minimize the loss.
    optimizer.step()



step          0 | train loss 10.8203 | val loss 10.8197
step         50 | train loss 4.5970 | val loss 4.4879
step        100 | train loss 3.2229 | val loss 3.2252
step        150 | train loss 2.9052 | val loss 2.8946
step        200 | train loss 2.8765 | val loss 2.7437
step        250 | train loss 2.7470 | val loss 2.7874
step        300 | train loss 2.7590 | val loss 2.7351
step        350 | train loss 2.7120 | val loss 2.6929
step        400 | train loss 2.6833 | val loss 2.6397
step        450 | train loss 2.6882 | val loss 2.6933
step        499 | train loss 2.6578 | val loss 2.6486


## Save Model Checkpoint

In [13]:
save_model_to_chekpoint(model = m, path_to_checkpoint="checkpoint", epoch=step)

Successfully saved the model to checkpoint/checkpoint_epoch-499_24.03.2023_09:37:06.pt


## Generate Output Samples

In [ ]:
# generate some output based on the context
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)

for i in range(10):
    print(f"-------------Output Example {i}----------------")
    print(
        decode(
            enc_sec=m.generate(idx=context, max_new_tokens=100, block_size=BLOCK_SIZE)[0],
            tokenizer=tokenizer,
        )
    )

In [17]:
# generate 10 input sentences
sentences = ['This is the sound of Music',
             'Do Re me fa so la ti do.',
             'Do a deer a female deer.',
             'Ray a drop of golden sun.',
             'Me a name I call myself.',
             'Let us move sentence six.',
             'Fa a long long way to run.',
             'do ti la so fa me re do.',
             'When I feel like I am alone.',
             'I sing this song of my favourite things.']

# tokenize the input sentences
tokenized_sentences = [tokenizer.encode(sentence) for sentence in sentences]
i=1
# generate output for each sentence
for sentence in tokenized_sentences:
    print(f"-------------Output Example {i}----------------")
    context = torch.tensor(sentence, dtype=torch.long, device=DEVICE).unsqueeze(0)
    output_tokens = m.generate(idx=context, max_new_tokens=35, block_size=BLOCK_SIZE)[0]
    output_sentence = decode(enc_sec=output_tokens, tokenizer=tokenizer)
    print(output_sentence)
    i=i+1




-------------Output Example 1----------------
[CLS] this is the sound of music [SEP] average. the second - gallup, giving vast quantities of government that number of the first wave of the united states holds jurisdiction that the four territories. 6 have won the five
-------------Output Example 2----------------
[CLS] do re me fa so la ti do. [SEP] caricaid. on their diet by france and the british army to western migration to be the country. military and the creation of world war ii, or their prison policy accounts
-------------Output Example 3----------------
[CLS] do a deer a female deer. [SEP] a situation that does not ratify the second term in the communist forces of those reside. of reconstruction began the second british army at the u. citizens and measures to the 1846
-------------Output Example 4----------------
[CLS] ray a drop of golden sun. [SEP] the three - highest total - de facto national guard brought the closest ally of 2020. the soviet union as of became increasingly 